In [1]:
import pandas as pd
import json

c:\Users\USER\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [30]:
f = open(r'C:\Users\USER\Desktop\Learning\PyStrava\Outputs\all_workouts.json')
workout_json = json.load(f)

# we have to recreate this in the time_minutes column 
# because we21.92 - (21.92%1)) + (((21.92%1) * 60)/100)
df = pd.json_normalize(workout_json)
df['distance'] = round(df['distance']/1000,2)
df['workout_time_min'] = round(df['moving_time']/60,2)
df['workout_time_min'] = round(df['workout_time_min'] - (df['workout_time_min']%1) + (((df['workout_time_min']%1) * 60)/100),2)
df["start_date"] = pd.to_datetime(df['start_date']).dt.date
df['average_speed'] = df['average_speed'] * 3.6
df['max_speed'] = df['max_speed'] * 3.6
df = df.rename(columns={'id':'activity_id','average_speed':'average_speed_km/h','max_speed':'max_speed_km/h'})
df['sport_type'] = df['sport_type'].replace({'Workout':'Functional-Cardio Workout'})
# Creating the start and end latitude and longitude
df[['start_lat', 'start_long']] = df['start_latlng'].apply(lambda x: pd.Series(str(x).strip('[]').split(',')))
df[['end_lat', 'end_long']] = df['end_latlng'].apply(lambda x: pd.Series(str(x).strip('[]').split(',')))

df = df[['activity_id',
            'name',
            'start_date',
            'sport_type',
            'distance',
            'workout_time_min',
            'calories',
            'total_elevation_gain',
            'start_lat',
            'start_long',
            'end_lat',
            'end_long',
            'average_speed_km/h',
            'max_speed_km/h',
            'average_temp',
            'average_heartrate',
            'max_heartrate']]



workout_laps = pd.json_normalize(workout_json,'laps')
workout_laps = workout_laps[['activity.id','name','moving_time','distance','average_heartrate','max_heartrate','average_speed','max_speed']]
workout_laps = workout_laps.rename(columns={'activity.id':'activity_id',
                                                'name':'lap',
                                                'moving_time':'lap_time_min',
                                                'distance':'lap_distance',
                                                'average_heartrate':'lap_average_heartrate',
                                                'max_heartrate':'lap_max_heartrate_km/h',
                                                'average_speed':'lap_average_speed_km/h',
                                                'max_speed':'lap_max_speed'})
workout_laps['lap_time_min'] = round(workout_laps['lap_time_min']/60,2)
workout_laps['lap_time_min'] = round(workout_laps['lap_time_min'] - (workout_laps['lap_time_min']%1) + (((workout_laps['lap_time_min']%1) * 60)/100),2)
workout_laps['lap_distance'] = round(workout_laps['lap_distance']/1000,2)

# Getting the average lap time for each workout
avg_lap_time = workout_laps.pivot_table(index=['activity_id'],values = 'lap_time_min',aggfunc='mean')

In [27]:
workout_laps.to_csv(r'C:\Users\USER\Desktop\Learning\PyStrava\Outputs\workout_laps_test.csv')

In [29]:
avg_lap_time = workout_laps.pivot_table(index=['activity_id'],values = 'lap_time_min',aggfunc='mean')
avg_lap_time = avg_lap_time.reset_index()

,lap_time_min
activity_id,
9085384121,7.243750
9089552781,8.697500
9097521399,6.555000
9102729006,5.803333
9116553075,5.180000
...,...
9893694843,4.138000
9919975714,3.410000
9927103494,7.105000


In [26]:
# avg_time_per_lap = workout_laps.groupby('activity_id').mean()

In [ ]:
avg_time_per_lap = avg_time_per_lap.reset_index()
avg_time_per_lap = avg_time_per_lap[['activity_id','lap_time_min']]
avg_time_per_lap = avg_time_per_lap.rename(columns={'lap_time_min':'avg_time_per_lap'})
avg_time_per_lap = workout_laps.groupby('activity_id').mean()


lap_counter = workout_laps['activity_id'].value_counts().rename_axis('activity_id').reset_index(name='lap_count')

lap_stats = avg_time_per_lap.merge(lap_counter,on='activity_id')

merged = df.merge(lap_stats, on = 'activity_id')
merged['lap_count'] = pd.to_numeric(merged['lap_count'])

In [18]:
workout_laps['lap_time_min'] = round(workout_laps['lap_time_min']/60,2)
workout_laps['lap_time_min'] = round(workout_laps['lap_time_min'] - (workout_laps['lap_time_min']%1) + (((workout_laps['lap_time_min']%1) * 60)/100),2)
workout_laps['lap_distance'] = round(workout_laps['lap_distance']/1000,2)

In [19]:
workout_laps

,activity_id,lap,lap_time_min,lap_distance,lap_average_heartrate,lap_max_heartrate_km/h,lap_average_speed_km/h,lap_max_speed
0,9939802653,Lap 1,6.22,1.00,132.9,153.0,2.62,3.102
1,9939802653,Lap 2,5.55,1.00,156.4,163.0,2.82,4.337
2,9939802653,Lap 3,5.57,1.00,159.7,164.0,2.80,3.767
3,9939802653,Lap 4,5.39,1.00,161.5,167.0,2.95,5.264
4,9939802653,Lap 5,6.07,1.00,163.2,166.0,2.72,3.938
...,...,...,...,...,...,...,...,...
761,9085384121,Lap 12,7.35,1.00,149.1,158.0,2.20,3.028
762,9085384121,Lap 13,7.34,1.00,147.2,152.0,2.20,3.256
763,9085384121,Lap 14,8.01,1.00,146.0,166.0,2.08,3.162
764,9085384121,Lap 15,8.02,1.00,145.3,154.0,2.07,2.974


In [ ]:
# pending => fix this 
workout_laps['lap_time_min'] = round(workout_laps['lap_time_min']/60,2)
workout_laps['lap_time_min'] = round(workout_laps['lap_time_min'] - (workout_laps['lap_time_min']%1) + (((workout_laps['lap_time_min']%1) * 60)/100),2)

In [117]:
workout_laps

,activity_id,lap,lap_time_min,lap_distance,lap_average_heartrate,lap_max_heartrate_km/h,lap_average_speed_km/h,lap_max_speed
0,9939802653,Lap 1,382,1000.00,132.9,153.0,2.62,3.102
1,9939802653,Lap 2,355,1000.00,156.4,163.0,2.82,4.337
2,9939802653,Lap 3,357,1000.00,159.7,164.0,2.80,3.767
3,9939802653,Lap 4,339,1000.00,161.5,167.0,2.95,5.264
4,9939802653,Lap 5,367,1000.00,163.2,166.0,2.72,3.938
...,...,...,...,...,...,...,...,...
761,9085384121,Lap 12,455,1000.00,149.1,158.0,2.20,3.028
762,9085384121,Lap 13,454,1000.00,147.2,152.0,2.20,3.256
763,9085384121,Lap 14,481,1000.00,146.0,166.0,2.08,3.162
764,9085384121,Lap 15,482,1000.00,145.3,154.0,2.07,2.974


## Trouble shooting laps
--------------------------

In [94]:
file_name = r"C:\Users\USER\Desktop\Learning\PyStrava\Outputs\all_workouts.json"

with open(file_name, "r") as file:
    laps_json = json.load(file)

In [95]:
workout_laps = pd.json_normalize(laps_json,'laps')
workout_laps

,id,resource_state,name,elapsed_time,moving_time,start_date,start_date_local,distance,start_index,end_index,...,average_heartrate,max_heartrate,lap_index,split,pace_zone,activity.id,activity.visibility,activity.resource_state,athlete.id,athlete.resource_state
0,34059985483,2,Lap 1,443,382,2023-09-28T21:31:24Z,2023-09-28T16:31:24Z,1000.00,0,76,...,132.9,153.0,1,1,0.0,9939802653,everyone,1,58832723,1
1,34059985485,2,Lap 2,355,355,2023-09-28T21:38:50Z,2023-09-28T16:38:50Z,1000.00,77,160,...,156.4,163.0,2,2,0.0,9939802653,everyone,1,58832723,1
2,34059985486,2,Lap 3,357,357,2023-09-28T21:44:47Z,2023-09-28T16:44:47Z,1000.00,161,236,...,159.7,164.0,3,3,0.0,9939802653,everyone,1,58832723,1
3,34059985487,2,Lap 4,355,339,2023-09-28T21:50:44Z,2023-09-28T16:50:44Z,1000.00,237,313,...,161.5,167.0,4,4,0.0,9939802653,everyone,1,58832723,1
4,34059985489,2,Lap 5,367,367,2023-09-28T21:56:38Z,2023-09-28T16:56:38Z,1000.00,314,396,...,163.2,166.0,5,5,0.0,9939802653,everyone,1,58832723,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
761,30932547350,2,Lap 12,468,455,2023-05-17T01:11:40Z,2023-05-16T20:11:40Z,1000.00,977,1080,...,149.1,158.0,12,12,0.0,9085384121,everyone,1,58832723,1
762,30932547351,2,Lap 13,507,454,2023-05-17T01:19:30Z,2023-05-16T20:19:30Z,1000.00,1081,1180,...,147.2,152.0,13,13,0.0,9085384121,everyone,1,58832723,1
763,30932547352,2,Lap 14,481,481,2023-05-17T01:27:58Z,2023-05-16T20:27:58Z,1000.00,1181,1286,...,146.0,166.0,14,14,0.0,9085384121,everyone,1,58832723,1
764,30932547353,2,Lap 15,482,482,2023-05-17T01:35:58Z,2023-05-16T20:35:58Z,1000.00,1287,1366,...,145.3,154.0,15,15,0.0,9085384121,everyone,1,58832723,1
